# SBS Maltese Food Recipes
---

In [3]:
# Libraries needed to get the html from a site and to parse the html
from bs4 import BeautifulSoup as bs
import requests
import csv
from parse_ingredients import parse_ingredient
import pyfood as pyf

In [4]:
# Saving the url to a variable, getting the url and parsing the html
mainURL = "https://www.sbs.com.au/food/cuisine/maltese"
req = requests.get(mainURL)
soup = bs(req.text, "html.parser")

In [5]:
# Base link that hrefs will be added onto
baseLink = 'https://www.sbs.com.au'

In [6]:
# The tag that stores the href to the recipes
hrefs = soup.findAll(class_='link-underlay')

In [7]:
# Simply printing out the contents to see what the hrefs of the tags are

def printHrefs(hrefsIn):
    for i in hrefsIn:
        print(i.get('href'), "\n")

# printHrefs(hrefs)

In [8]:
# This for loop will generate the links to all the recipes that are shown on the mainURL

def funcRecipeLinks(hrefsIn, baseLinkIn, recipeLinks):
    for i in hrefsIn:
        fullLink = baseLinkIn + i.get('href')
        if i.get('href')[6] == 'r':
            recipeLinks.append(fullLink)
    return recipeLinks

recipeLinks = []
recipeLinks = funcRecipeLinks(hrefs, baseLink, recipeLinks)

In [9]:
mainURL = "https://www.sbs.com.au/food/cuisine/maltese?sort_by=created&page=1"
req = requests.get(mainURL)
soup = bs(req.text, "html.parser")

baseLink = 'https://www.sbs.com.au'
hrefs = soup.findAll(class_='link-underlay')

recipeLinks = funcRecipeLinks(hrefs, baseLink, recipeLinks)


In [10]:
# Printing the links to all the recipes shown before clicking the show more button
print(recipeLinks)
print(len(recipeLinks))

['https://www.sbs.com.au/food/recipes/maltese-kapunata', 'https://www.sbs.com.au/food/recipes/maltese-baked-ricotta-pie', 'https://www.sbs.com.au/food/recipes/maltese-style-bruschetta-hobz-biz-zejt', 'https://www.sbs.com.au/food/recipes/maltese-rabbit-stew-stuffat-tal-fenek', 'https://www.sbs.com.au/food/recipes/bread-oil-hobz-biz-zejt', 'https://www.sbs.com.au/food/recipes/maltese-easter-biscuits-figolli', 'https://www.sbs.com.au/food/recipes/timpana', 'https://www.sbs.com.au/food/recipes/broad-bean-dip-bigilla', 'https://www.sbs.com.au/food/recipes/pears-baked-white-wine-almonds-and-armaretto-langas-mimli-bil-lewz', 'https://www.sbs.com.au/food/recipes/apple-and-fig-cake', 'https://www.sbs.com.au/food/recipes/pork-and-pea-pastizzi', 'https://www.sbs.com.au/food/recipes/rabbit-tomato-and-wine-sauce-rice', 'https://www.sbs.com.au/food/recipes/peach-and-almond-crumble', 'https://www.sbs.com.au/food/recipes/christmas-chocolate-salami-romblu-tal-milied-bic-cikkulata', 'https://www.sbs.com

## Testing different ingredient extraction methods
---
### Method 1: parse_ingredients library

The parse_ingredients library provides a way to extract ingredients, quantity, units and comments from recipe ingredients. In testing it was found that the library can be inconsistent with removing quantifiers such as size, colour and plurals. 

In [ ]:
def parseIngredientsFunc(i, title):
    data = []
    for j in title:
                data.append(str(j.get_text(strip=False))) # Dish Name
    
    parseResult = parse_ingredient(str(i.get_text(strip=False)))
            
    data.append(parseResult.original_string) #Original String
    data.append(parseResult.name) #Ingredient
    data.append(parseResult.quantity) #Quantity
    data.append(parseResult.unit) #Unit
    data.append(parseResult.comment) #Comment
    
    return data

### Method 2: pyfood library

Contrary to the parse_ingredient library, the pyfood library only extracts the ingredient from the input string. Moreover, It is prone to some silly errors where it will convert ingredients like 'frozen peas' to 'green peppers'. These mistakes appear to be few and far between. However, examination shows a better handling of quantifiers; it ignores indicators to size and colour, as well as removes plurals. 

In [ ]:
def pyfoodFunc(title, i):
    data = []

    for j in title:
        data.append(str(j.get_text(strip=False))) # Dish Name
        
    print(str(i.get_text(strip=False)))
    results = shelf.process_ingredients([i.get_text(strip=False)])
    try:
        temp = results['ingredients'][0]['foodname'] # vegetarian, vegan, nutrition, seasonality
    except:
        temp = results['HS'][0]
    data.append(i)
    data.append(temp)

    return data

### Method 3: parse_ingredients and pyfood libraries

After taking into consideration the above scenario, we decided to use both of these libraries for the best result. First, the parse_ingredients library is used to extract: quantity, and the ingredient name. The ingredient name is then passed trough the pyfoods function set to obtain a striped down version of the ingredient. Using the parse_ingredients library first also appears to reduce the error in the pyfoods library. 

In [96]:
def parseIngredientsAndPyfoodFunc(i, title):
    data = []
    for j in title:
                data.append(str(j.get_text(strip=False))) # Dish Name
    
    parseResult = parse_ingredient(str(i.get_text(strip=False)))

    results = shelf.process_ingredients([parseResult.name])
    try:
        temp = results['ingredients'][0]['foodname'] # vegetarian, vegan, nutrition, seasonality
    except:
        temp = results['HS'][0]
    data.append(str(i.get_text(strip=False)))
    data.append(temp)

    return data

In [ ]:
ingredientList = []
shelf = pyf.Shelf(region='Italy', month_id=0)

for recipe in recipeLinks:
    req = requests.get(recipe) # Accesses the next recipe
    soup = bs(req.text, "html.parser")
    title = soup.findAll('h1') # finds the name of the recipe
    
    ingredientsDiv = soup.findAll('div', class_='field-name-field-ingredients') # finds all the divs containing the ingredients
    
    # For each div go trough and extract the ingredients
    for ul in ingredientsDiv:
        ingredients = ul.findAll("li")
        
        for i in ingredients:
            # data = parseIngredientsFunc(i, title)
            # data = pyfoodFunc(title, i)
            data = parseIngredientsAndPyfoodFunc(i, title)
            
            ingredientList.append(data)

In [21]:
import csv

header = ['Dish Name', 'Original String', 'Ingredient', 'Quantity', 'Unit', 'Comment']


with open('CSV/sbsRecipes.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(ingredientList)

In [1]:
def saveToCSV(header, rows, filePath):
    with open(filePath, 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)

        # write the header
        writer.writerow(header)

        # write multiple rows
        writer.writerows(rows)

In [98]:
header = ['Dish Name', 'Original String', 'Ingredient']

with open('CSV/sbsRecipes3.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(ingredientList)

# Save recipes and Links to CSV

The following for loop looks over all the links previously scraped, gets the title from each page (this will indicate the recipe name), and saves the recipe name and link to csv

In [15]:
RecipeList = []

for recipe in recipeLinks:
    req = requests.get(recipe) # Accesses the next recipe
    soup = bs(req.text, "html.parser")
    title = soup.findAll('h1') # finds the name of the recipe

    row = []

    for j in title:
        row.append(str(j.get_text(strip=False)))

    row.append(recipe)

    RecipeList.append(row)

In [13]:
header = ['Recipe Name', 'Links']

saveToCSV(header, RecipeList, '../CSV/listOfRecipes.csv')